In [28]:
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import os
import os.path

In [29]:
def scrap(link):
    if 'https://' in link:
        link = link.replace('https://', '')
    if 'http://' in link:
        link = link.replace('http://', '')
        
    # scrap
    key = '1783319260d70f84da21868ce0fd6207'
    api_link = f"http://api.scraperapi.com?api_key={key}&url={link}"
    page=urlopen(api_link)
    return page

In [30]:
def write(page, category):
    # Read data on page
    html_bytes=page.read()
    # Decode data from page
    html=html_bytes.decode("utf-8")
    # Write html file from data scrapped
    with open(f"html_dir/{category}.html","w",encoding="utf-8")as html_file:
        html_file.write(html)

In [42]:
def read(category):    
    # Read html file from scrapped data
    with open(f"html_dir/{category}.html","r",encoding='utf-8')as html_file:
        output=html_file.read()
        
        soup = BeautifulSoup(output, 'html')
        print("_________________________________________________")
        players = soup.find_all('tr', class_ = 'data-v-6a4dcc8d')
        for player in players:
            print(player)
       
        # product_dict = {}
        # for product in products:
        #     link = str('komplett.no' + product.a['href'])
        #     image = str('komplett.no/img/p/800/' + product.a['href'].split('/')[2] + '.jpg')
        #     sale = None
        #     price_now = product.find('span', class_= 'product-price-now').text
        #     price_now = int(''.join(char for char in price_now if char.isalnum()))
        #     price_before = product.find('div', class_= 'product-price-before')
        #     if price_before != None:
        #         price_before = product.find('div', class_= 'product-price-before').text.replace('Før', '').replace(',-', '').strip()
        #         price_before = int(''.join(char for char in price_before if char.isalnum()))
        #         sale = True
            
        #     name = product.h2.text.replace('å', 'aa').replace('ø', 'o')
        #     stats = product.p.text.replace(',','').split()
        #     #available = product.find('span', class_='stockstatus-stock-details').text.replace('å', 'aa').replace('ø', 'o').split(' ')[0]
        #     try:
        #         available = product.find('span', class_='stockstatus-stock-details').text.replace('å', 'aa').replace('ø', 'o').split(' ')[0].replace('+', '')
        #         if available == 'Ikke' or available == 'Bestillingsvare.':
        #             available = 0
        #     except AttributeError as error:
        #             available = error
                    
        #     itemnumber = product.find('div', class_='product-data').text.replace(' ', '').replace('\n', '').split('/')[0].split(':')[1]

        #     product_dict[itemnumber] = [name, price_now, price_before, sale, available, stats, image, link, 'komplett', category]       
        
        # return product_dict

In [43]:
def komplett_scrap(mode):

    # Check if we have the correct directories
    if not os.path.exists('html_dir'):
        os.mkdir('html_dir')
    if not os.path.exists('excel_dir'):
        os.mkdir('excel_dir')

    link = f"https://tracker.gg/valorant/{mode}"
    scrapped = scrap(link)
    write(scrapped, mode)
    read(mode)

In [44]:
komplett_scrap("lfg")

_________________________________________________


In [34]:
        # Category on the item in our current link
        # category = link.split('?')[0].split('/')[-1].replace('-','_')
        # print(category)
        # # Write down the data we get from scrapping
        # scrapped = scrap(link, category, tries)
        # if scrapped != None:
        #     print('\tScrap: Success!')
        #     print('\tWriting...')
        #     write(scrapped, category)
        #     print('\tReading...')
        #     current = read(category)
        #     # Used for columns in excel
        #     col = ['name', 'price_now', 'price_before', 'sale', 'available', 'stats', 'image', 'link', 'site', 'category']
        #     # Create dataframe
        #     df = pd.DataFrame(current.values(), columns = col)
        #     #df = df.loc[df['sale']==True]
        #     # Convert dataframe to excel file
        #     df.to_csv(f'excel_dir/{category}.csv')
        #     print(f'\tScrapped: {category}')